# Libraries

In [2]:
import pandas as pd
import json

# Load Data

#### Match Results

In [3]:
# Match results
with open(r'C:\Users\Usuario\Portfolio\Web-Scrapping-Visualization-PowerBI\data\raw\t20_json_files\t20_wc_match_results.json') as f:
    data = json.load(f)

df_match = pd.DataFrame(data[0]['matchSummary'])

#### Batting Summary

In [ ]:
# Batting summary
with open(r'C:\Users\Usuario\Portfolio\Web-Scrapping-Visualization-PowerBI\data\raw\t20_json_files\t20_wc_batting_summary.json') as f:
    data_1 = json.load(f)

    all_records = []

    for rec in data_1:
        all_records.extend(rec['battingSummary'])

df_summary = pd.DataFrame(all_records)

#### Bowling Summary

In [23]:
with open(r'C:\Users\Usuario\Portfolio\Web-Scrapping-Visualization-PowerBI\data\raw\t20_json_files\t20_wc_bowling_summary.json') as f:
    data_2 = json.load(f)

    all_records_2 = []

    for rec in data_2:
        all_records_2.extend(rec['bowlingSummary'])

df_bowling = pd.DataFrame(all_records_2)

#### Player Information

In [31]:
with open(r'C:\Users\Usuario\Portfolio\Web-Scrapping-Visualization-PowerBI\data\raw\t20_json_files\t20_wc_player_info.json') as f:
    data_3 = json.load(f)

df_players = pd.DataFrame(data_3)

# Explore and transform Data

#### Match Results

In [11]:
df_match.head()

,team1,team2,winner,margin,ground,matchDate,match_id
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


In [5]:
# Rename columns
df_match.rename({'scorecard' : 'match_id'}, axis = 1, inplace=True)

In [10]:
# Create dictionary

match_ids_keys = {}

for index, row in df_match.iterrows():
    key_1 = row['team1'] + ' Vs ' + row['team2']
    key_2 = row['team2'] + ' Vs ' + row['team1']

    match_ids_keys[key_1] = row['match_id']
    match_ids_keys[key_2] = row['match_id']


In [21]:
df_match.to_csv('../data/processed/df_match_results.csv', index=False)

#### Batting Summary

In [6]:
df_summary.head()

,match,teamInnings,battingPos,batsmanName,dismissal,runs,balls,4s,6s,SR
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,c Pramod Madushan b Chameera,3,6,0,0,50.00
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,c Shanaka b Pramod Madushan,9,9,1,0,100.00
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,c †Mendis b Karunaratne,20,12,1,2,166.66
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,c DM de Silva b Pramod Madushan,26,24,2,0,108.33
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),c Gunathilaka b PWH de Silva,20,24,0,0,83.33


In [7]:
# Add feature
df_summary['out/not_out'] = df_summary.dismissal.apply(lambda x : 'out' if len (x) > 0 else 'not out')

In [8]:
# Drop columns
df_summary.drop(['dismissal'], axis = 1, inplace=True)

In [9]:
# Remove special characters
df_summary['batsmanName'] = df_summary['batsmanName'].apply(lambda x : x.replace('†',''))

In [12]:
# Add match_id

df_summary['match_id'] = df_summary['match'].map(match_ids_keys)

In [20]:
df_summary.to_csv('../data/processed/df_batting_summary.csv', index = False)

#### Bowling Sumary

In [26]:
df_bowling['match_id'] = df_bowling['match'].map(match_ids_keys)

In [28]:
df_bowling.to_csv('../data/processed/df_bowling_summary.csv', index = False)

#### Players Information

In [34]:
df_players['name']= df_players['name'].apply(lambda x : x.replace('†',''))

In [39]:
df_players.to_csv('../data/processed/df_players_information.csv', index = False)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/processed/df_players_information.csv')

In [3]:
# change name jan nicol loftie to jan nicol
df[df['name'] == 'Jan Nicol Loftie-Eaton'] = 'Jan Nicol'

In [4]:
df['name'] = df['name'].apply(lambda x : x.replace('(c)', ''))

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


In [6]:
driver = webdriver.Chrome()

In [7]:
driver.get("https://www.espncricinfo.com")

In [8]:
WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))).click()

In [9]:
contador_consentimiento = 0

def obtener_url_imagen_jugador():
    global contador_consentimiento
    for index, row in df.iterrows():
        nombre_jugador = row['name']
        driver.get("https://www.espncricinfo.com")
        try:
            search_icon = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "i.icon-search-outlined")))
            search_icon.click()

            search_box_selector = "#main-container > div:nth-child(3) > div > nav > div > div > div > div.ds-flex.ds-items-center.ds-justify-between.ds-flex-1 > div.ds-flex.ds-flex-row.ds-items-center > div > form > div > div > div > input"
            search_box = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, search_box_selector)))
            
            search_box.clear()
            search_box.send_keys(nombre_jugador + Keys.RETURN)  # Reemplaza "Nombre del Jugador" con el nombre real que estás buscando

            
            if contador_consentimiento == 0:
                consent_button_selector = "body > div.fc-consent-root > div.fc-dialog-container > div.fc-dialog.fc-choice-dialog > div.fc-footer-buttons-container > div.fc-footer-buttons > button.fc-button.fc-cta-consent.fc-primary-button"
                consent_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, consent_button_selector)))
                consent_button.click()
                contador_consentimiento += 1  # Incrementa el contador después de hacer clic


            player_link_selector = "#viewport > div.hp-wrapper > div.hp-container.search > main > div > div.search-cat-links > ul > li:nth-child(2) > a"
            players_link = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, player_link_selector)))
            players_link.click()

            player_profile_link_selector = "#viewport > div.hp-wrapper > div.hp-container.search > main > div > div.results.in-players > ul > li:nth-child(2) > h3 > a"
            player_profile_link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, player_profile_link_selector)))
            player_profile_link.click()


            player_image_selector = "#main-container > div.ds-relative > div.lg\\:ds-container.lg\\:ds-mx-auto.lg\\:ds-px-5.ds-pt-4 > div.ds-flex.ds-space-x-5 > div.ds-flex-none > div:nth-child(1) > div > div > div > div.ds-bg-cover.ds-bg-center > div > div:nth-child(2) > div > div > img"
    
            # Esperar hasta que la imagen del jugador esté presente en la página y luego obtener su atributo 'src'
            player_image = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, player_image_selector)))
            df.at[index, 'image_url'] = player_image.get_attribute('src')

        except Exception as e:
            print(f"Error al obtener la imagen para {nombre_jugador}: {e}")
            df.at[index, 'image_url'] = None






In [10]:
obtener_url_imagen_jugador()

Error al obtener la imagen para Michael van Lingen: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF642B85E42+3538674]
	(No symbol) [0x00007FF6427A4C02]
	(No symbol) [0x00007FF642655AEB]
	(No symbol) [0x00007FF64269BF4E]
	(No symbol) [0x00007FF64269C0CC]
	(No symbol) [0x00007FF6426DE477]
	(No symbol) [0x00007FF6426BF0EF]
	(No symbol) [0x00007FF6426DBDE3]
	(No symbol) [0x00007FF6426BEE53]
	(No symbol) [0x00007FF64268F514]
	(No symbol) [0x00007FF642690631]
	GetHandleVerifier [0x00007FF642BB6CAD+3738973]
	GetHandleVerifier [0x00007FF642C0C506+4089270]
	GetHandleVerifier [0x00007FF642C04823+4057299]
	GetHandleVerifier [0x00007FF6428D5C49+720121]
	(No symbol) [0x00007FF6427B126F]
	(No symbol) [0x00007FF6427AC304]
	(No symbol) [0x00007FF6427AC432]
	(No symbol) [0x00007FF64279BD04]
	BaseThreadInitThunk [0x00007FFEFC03257D+29]
	RtlUserThreadStart [0x00007FFEFE12AA58+40]

Error al obtener la imagen para Wanindu Hasaranga de Silva: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF642B85E42+

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.140)
Stacktrace:
	GetHandleVerifier [0x00007FF642B85E42+3538674]
	(No symbol) [0x00007FF6427A4C02]
	(No symbol) [0x00007FF642655AEB]
	(No symbol) [0x00007FF64263288C]
	(No symbol) [0x00007FF6426C5DD7]
	(No symbol) [0x00007FF6426DB40F]
	(No symbol) [0x00007FF6426BEE53]
	(No symbol) [0x00007FF64268F514]
	(No symbol) [0x00007FF642690631]
	GetHandleVerifier [0x00007FF642BB6CAD+3738973]
	GetHandleVerifier [0x00007FF642C0C506+4089270]
	GetHandleVerifier [0x00007FF642C04823+4057299]
	GetHandleVerifier [0x00007FF6428D5C49+720121]
	(No symbol) [0x00007FF6427B126F]
	(No symbol) [0x00007FF6427AC304]
	(No symbol) [0x00007FF6427AC432]
	(No symbol) [0x00007FF64279BD04]
	BaseThreadInitThunk [0x00007FFEFC03257D+29]
	RtlUserThreadStart [0x00007FFEFE12AA58+40]


In [11]:
df.isnull().sum()

name              0
team              0
battingStyle      0
bowlingStyle     20
playingRole       0
description      67
image_url       106
dtype: int64